In [1]:
import requests
import pandas as pd
import json
import helper

pd.set_option('display.max_rows', None)

weiDivisible = 1000000000000000000.0
tokens = pd.read_csv("dataCollection_TOKENLIST.csv")

for token in tokens['Contract']:
    tokenContract = token
    url = f"https://api.ethplorer.io/getTopTokenHolders/{tokenContract}?apiKey=freekey&limit=50"

    response = requests.get(url)
    response = response.json()
    response = response['holders']
    #data = json.loads(response)

    df = pd.DataFrame.from_dict(response)

    addresses = df['address']
    #if(len(addresses) <= 20): continue
    isContract = []
    for address in addresses:
        if(helper.isContract(address) or address == "0x000000000000000000000000000000000000dead"):
            isContract.append(True)
        else: 
            isContract.append(False)
        
    df["isContract"] = isContract
    df2 = df[df["isContract"] == False]
    df = df[df["isContract"] == True]
    df.to_csv("result_contracts.csv")

    addresses = df2["address"]
    ethBalances = []
    tokenBalances = []
    for address in addresses:
        url = f"https://api.ethplorer.io/getAddressInfo/{address}?apiKey=EK-jCkqf-qQHnUdG-uyNuL"

        response = requests.get(url)
        response = response.json()

        ethBalance = response['ETH']['balance'] * response['ETH']['price']['rate']
        tokenBalance = helper.getTokenBalance(address)

        ethBalances.append(ethBalance)
        tokenBalances.append(tokenBalance)

    df2['EthBalance'] = ethBalances
    df2['TokenBalance'] = tokenBalances
    df2["EthBalance"] = pd.to_numeric(df2["EthBalance"])
    df2["TokenBalance"] = pd.to_numeric(df2["TokenBalance"])
    df2['TotalBalance'] = df2["EthBalance"] + df2["TokenBalance"]

    #   RESEARCH FINDINGS
    # Fresh: Total Transaction <= 10 
    # Good:  Daily Transaction <= 5 (Inactive wallets also 'Good')
    # Aight: Daily Transaction 5 < x <= 10
    # Jett:   Daily Transaction > 10

    wallets = df2["address"]
    walletLabel = []
    goodWalletsToAnalyzeLater = []

    for wallet in wallets:
        url=f"https://api.etherscan.io/api?module=account&action=tokentx&address={wallet}&page=1&offset=100&startblock=0&endblock=99999999&sort=desc&apikey=7YRRHUFSDSM5BXCIM8FY3NPJ192WYKRC1G"

        resp=requests.get(url)
        resp=resp.json()
        resp=resp['result']

        if len(resp) < 10: 
            walletLabel.append("Fresh")
            continue

        yesterdayTime = int(resp[0]['timeStamp']) - 86400
        walletTransfers = []
        for transfer in resp:
            if int(transfer['timeStamp']) >= yesterdayTime: walletTransfers.append(transfer['timeStamp'])

        walletTransfers = [*set(walletTransfers)]
        dailyTransfer = len(walletTransfers)

        if dailyTransfer <= 5: 
            walletLabel.append("Good")
            goodWalletsToAnalyzeLater.append(wallet)
        elif dailyTransfer > 5 and dailyTransfer <= 10: walletLabel.append("Aight")
        if dailyTransfer > 10: walletLabel.append("Jett")

    #with open("testDEJITARUPARUGoodWallets.csv", "w") as f:
    #    for wallet in goodWalletsToAnalyzeLater:
    #        f.write(wallet + "\n")

    df2['Label'] = walletLabel
    df2.to_csv("result.csv")

    

    df = pd.read_csv("result.csv")
    df2 = pd.read_csv("result_contracts.csv")

    walletAmount = len(df.index)
    contractsAmount = len(df2.index)

    shares = df.groupby(by="Label")["share"].sum()

    if 'Fresh' in shares and 'Good' in shares and 'Aight' in shares:
        walletsShare = shares['Aight'] + shares['Fresh'] + shares['Good'] + shares['Jett']
    elif 'Good' in shares and 'Aight' in shares:
        walletsShare = shares['Aight'] + shares['Good'] + shares['Jett']
    elif 'Good' in shares and 'Aight' not in shares:
        walletsShare = shares['Good'] + shares['Jett']
    elif 'Good' not in shares and 'Aight' in shares:
        walletsShare = shares['Aight'] + shares['Jett']
    else:
        walletsShare = shares['Jett']
    contractsShare = df2['share'].sum()

    if 'Fresh' in shares:
        freshAmount = (df['Label'].value_counts()['Fresh'])
    if 'Good' in shares:
        goodAmount = (df['Label'].value_counts()['Good'])
    if 'Aight' in shares:
        aightAmount = (df['Label'].value_counts()['Aight'])
    if 'Jett' in shares:
        jettAmount = (df['Label'].value_counts()['Jett'])

    if 'Fresh' in shares:
        freshShare = shares['Fresh']
    else: 
        freshShare = 0

    if 'Good' in shares:
        goodShare = shares['Good']
    else: 
        goodShare = 0

    if 'Aight' in shares:
        aightShare = shares['Aight']
    else: 
        aightShare = 0

    if 'Jett' in shares:
        JettShare = shares['Jett']
    else: 
        jettShare = 0
    

    megaRich = df[df['TotalBalance'] > 100000] 
    rich = df[(df['TotalBalance'] <= 100000) & (df['TotalBalance'] > 50000)] 
    comfy = df[(df['TotalBalance'] <= 50000) & (df['TotalBalance'] > 10000)] 
    decent = df[(df['TotalBalance'] <= 10000) & (df['TotalBalance'] > 1000)] 
    regular = df[df['TotalBalance'] <= 1000] 

    megaRichShare = megaRich['share'].sum()
    richShare = rich['share'].sum()
    comfyShare = comfy['share'].sum()
    decentShare = decent['share'].sum()
    regularShare = regular['share'].sum()

    if 'Fresh' in shares:
        shareScore = freshShare * 0.8 + goodShare * 1 + aightShare * 0.4 + JettShare * 0.1
    else:
        shareScore = goodShare * 1 + aightShare * 0.4 + JettShare * 0.1

    balanceScore = megaRichShare * 1 + richShare * 0.9 + comfyShare * 0.75 + decentShare * 0.5 + regularShare * 0.2
    timeNow = pd.Timestamp.now()

    with open("dataCollection_TEST.csv", "a") as f:
        f.write(f"{tokenContract},{shareScore},{balanceScore},{timeNow}\n")